In [97]:
from uqer import Client
from uqer import DataAPI as api
from QuantLib import *

In [98]:
_ = Client(token='f1b9bea1d0b4e489c5ab9b69c3e2326a1bee6057af858067dbd1546453f428b2')

16937@wmcloud.com 账号登录成功


In [99]:
maturity_date = Date(15, 9, 2018)
start_date = Date(15, 3, 2018)

spot = 1.

risk_free = 0.035
dividend_rate = 0.0
volatility = 0.50
day_count = Actual365Fixed()
calendar = China(China.SSE)

## Market Data
-------------------

In [100]:
calculation_date = start_date
Settings.instance().evaluationDate = calculation_date

spot_handle = RelinkableQuoteHandle(SimpleQuote(spot))
vol_handle =  RelinkableQuoteHandle(SimpleQuote(volatility))

flat_ts = YieldTermStructureHandle(FlatForward(calculation_date, risk_free, day_count))
dividend_yield = YieldTermStructureHandle(FlatForward(calculation_date, dividend_rate, day_count))
flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, calendar, vol_handle, day_count))

bsm_process = BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)

## Analytic Digital Option
-----------------------

### 1. 参数
-------------

In [101]:
strike = 1.1 * spot
cash_pay = 0.05 * spot

### 2. 计算
--------------

In [107]:
payoff = CashOrNothingPayoff(Option.Call, strike, cash_pay)
exercise = EuropeanExercise(maturity_date)
option = EuropeanOption(payoff, exercise)

In [108]:
%%time
engine = AnalyticEuropeanEngine(bsm_process)
option.setPricingEngine(engine)

bsm_price = option.NPV()
bsm_delta = option.delta()
print("BSM european digital theoreticl price is {0:.4f}%".format(bsm_price * 100.))
print("BSM european digital theoreticl delta is {0:.4f}%".format(bsm_delta * 100.))

BSM european digital theoreticl price is 1.6995%
BSM european digital theoreticl delta is 5.1037%
Wall time: 1.01 ms


## Analytic Spread Option
--------------------------

### 1. 参数
------------------

In [119]:
strike1 = 1.1 * spot
strike2 = 1.2 * spot

leverage = 0.9

### 2. 计算
-------------

In [120]:
payoff = GapPayoff(Option.Call, strike1, strike2)
exercise = EuropeanExercise(maturity_date)
option = EuropeanOption(payoff, exercise)

In [121]:
%%time
engine = AnalyticEuropeanEngine(bsm_process)
option.setPricingEngine(engine)

bsm_price = option.NPV() * leverage
bsm_delta = option.delta() * leverage
print("BSM european spread theoreticl price is {0:.4f}%".format(bsm_price * 100.))
print("BSM european spread theoreticl delta is {0:.4f}%".format(bsm_delta * 100.))

BSM european spread theoreticl price is 6.8093%
BSM european spread theoreticl delta is 34.3318%
Wall time: 999 µs


In [122]:
GapPayoff?